# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [7]:
#Import necessary Libraries 

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report

# Import models you're considering
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

## Model Selection

[For this revenue forecasting task, the most relevant model families are linear regression (high interpretability, strong baseline), tree ensembles like Random Forest/GBM (excellent for tabular nonlinear interactions), and neural networks (good when many engineered features interact in complex ways). We selected a fast-foward neural network (Keras Sequential MLP), defined with Dense hidden layers and Batch Normalization, as the primarz model. We prefered that model, because the feautres are tabular, but interaction heavy (e.g. calendar and weather effects), and MLP can learn nonlinear cross effects better than a linear model.]



[## Feature Engineering

## Feature Selection

Following `MF_20251113.ipynb`, we use features that are available at day/product-group level and strongly related to demand:

- **Calendar effects**: `IsWeekend`, `IsNewYears`, `IsHalloween`
- **Seasonality (Fourier terms)**: `sin_1y`, `cos_1y`, `sin_2y`, `cos_2y`
- **Event/holiday indicators**: `holiday`, `Easter`, `KielerWoche`
- **Autoregressive signals**: `Revenue_lag1`, `Revenue_lag7` (within each `Warengruppe`)
- **Product-group fixed effects**: one-hot encoded `Warengruppe` dummies

These features provide a strong, interpretable baseline before moving to more complex models.]


In [4]:
# Load and prepare data (aligned with MF_20251113.ipynb)
sales = pd.read_csv('/workspaces/TeamCPH/data/umsatzdaten_gekuerzt.csv', parse_dates=['Datum'])
wetter = pd.read_csv('/workspaces/TeamCPH/data/wetter1.csv', parse_dates=['Datum'])
kiwo = pd.read_csv('/workspaces/TeamCPH/data/kiwo.csv', parse_dates=['Datum'])
holidays = pd.read_csv('/workspaces/TeamCPH/data/school_holidays_SH.csv', parse_dates=['Datum'])

# Aggregate to daily revenue per product group
sales_daily = (
    sales
    .groupby(['Datum', 'Warengruppe'], as_index=False)['Umsatz']
    .sum()
)

# Merge exogenous data
merged = sales_daily.merge(wetter, on='Datum', how='left')
merged = merged.merge(kiwo, on='Datum', how='left')
merged = merged.merge(holidays, on='Datum', how='left')

# Ensure expected indicator columns exist
for col in ['holiday', 'Easter']:
    if col not in merged.columns:
        merged[col] = 0

if 'KielerWoche' in merged.columns:
    merged['KielerWoche'] = merged['KielerWoche'].fillna(0).astype(int)
else:
    merged['KielerWoche'] = 0

# Calendar features
merged = merged.sort_values(['Warengruppe', 'Datum']).reset_index(drop=True)
merged['IsWeekend'] = merged['Datum'].dt.weekday.isin([5, 6]).astype(int)
merged['IsNewYears'] = (merged['Datum'].dt.strftime('%m-%d') == '12-31').astype(int)
halloween_days = [f'10-{day:02d}' for day in range(24, 32)]
merged['IsHalloween'] = merged['Datum'].dt.strftime('%m-%d').isin(halloween_days).astype(int)

# Seasonality features (Fourier terms)
merged['DayOfYear'] = merged['Datum'].dt.dayofyear
merged['sin_1y'] = np.sin(2 * np.pi * merged['DayOfYear'] / 365.25)
merged['cos_1y'] = np.cos(2 * np.pi * merged['DayOfYear'] / 365.25)
merged['sin_2y'] = np.sin(4 * np.pi * merged['DayOfYear'] / 365.25)
merged['cos_2y'] = np.cos(4 * np.pi * merged['DayOfYear'] / 365.25)

# Lag features within product group
merged['Revenue_lag1'] = merged.groupby('Warengruppe')['Umsatz'].shift(1)
merged['Revenue_lag7'] = merged.groupby('Warengruppe')['Umsatz'].shift(7)

# Product-group dummies
wg_dummies = pd.get_dummies(merged['Warengruppe'], prefix='WG', drop_first=True, dtype=int)
merged = pd.concat([merged, wg_dummies], axis=1)

# Define predictors
predictors = [
    'holiday', 'Easter', 'KielerWoche',
    'IsWeekend', 'IsNewYears', 'IsHalloween',
    'sin_1y', 'cos_1y', 'sin_2y', 'cos_2y',
    'Revenue_lag1', 'Revenue_lag7',
] + wg_dummies.columns.tolist()

# Build modeling table and split
model_df = merged[['Umsatz'] + predictors].replace([np.inf, -np.inf], np.nan).dropna()
X = model_df[predictors]
y = np.log1p(model_df['Umsatz'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)


Train shape: (7433, 17)
Test shape: (1859, 17)


## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]


In [3]:
# Implement hyperparameter tuning
# Example using GridSearchCV with a DecisionTreeClassifier
# param_grid = {'max_depth': [2, 4, 6, 8]}
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [1]:
# Implement the final model (MLP from MF_neural_net_estimation)
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Load prepared features/labels from pickle files
subdirectory = "pickle_data"
training_features = pd.read_pickle(f"{subdirectory}/training_features.pkl")
validation_features = pd.read_pickle(f"{subdirectory}/validation_features.pkl")
training_labels = pd.read_pickle(f"{subdirectory}/training_labels.pkl")
validation_labels = pd.read_pickle(f"{subdirectory}/validation_labels.pkl")

model = Sequential([
    InputLayer(shape=(training_features.shape[1],)),
    BatchNormalization(),
    Dense(10, activation="relu"),
    Dense(4, activation="relu"),
    Dense(1),
])

model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))
history = model.fit(
    training_features,
    training_labels,
    epochs=20,
    validation_data=(validation_features, validation_labels),
)

model.save("python_model.h5")

2026-02-24 19:59:55.310923: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-24 19:59:56.987697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-24 20:00:01.446221: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Epoch 1/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 53609.8555 - val_loss: 51057.2188
Epoch 2/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 35961.2031 - val_loss: 19382.1230
Epoch 3/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13803.5625 - val_loss: 10415.2139
Epoch 4/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 8813.1094 - val_loss: 7293.7354
Epoch 5/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 6721.8071 - val_loss: 6413.4849
Epoch 6/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 6081.5562 - val_loss: 6084.4531
Epoch 7/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5808.1504 - val_loss: 5900.5698
Epoch 8/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5710.5884 - val_loss: 5717.9404
Epoch 9/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 5511.2075 - val_loss: 5434.3945
Epoch 10/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 5298.9395 - val_loss: 5211.6123
Epoch 11/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5234.6118 - 

## Evaluation Metrics

We evaluate performance using Mean Absolute Percentage Error (MAPE) and Mean Squared Error (MSE). MAPE is the primary metric because it is scale-free and easy to interpret as a percent error across product groups and time. MSE is tracked during training because it aligns with the loss function used to optimize the neural network and is sensitive to larger errors, which helps penalize large revenue misses. We report metrics on both training and validation sets to monitor generalization and detect overfitting.

In [ ]:
# Evaluate the model using your chosen metrics
# Example for classification
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))

# Example for regression
# mse = mean_squared_error(y_test, y_pred)

# Your evaluation code here

import numpy as np
import pandas as pd

# -----------------------------
# MAPE FUNCTION (SAFE VERSION)
# -----------------------------
def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    denom = np.maximum(np.abs(y_true), eps)
    return 100.0 * np.mean(np.abs(y_true - y_pred) / denom)


# -----------------------------
# PREDICTIONS
# -----------------------------
training_predictions = model.predict(training_features).reshape(-1)
validation_predictions = model.predict(validation_features).reshape(-1)

training_labels = np.asarray(training_labels).reshape(-1)
validation_labels = np.asarray(validation_labels).reshape(-1)


# -----------------------------
# OVERALL MAPE
# -----------------------------
print(f"MAPE on the Training Data: {mape(training_labels, training_predictions):.2f}%")
print(f"MAPE on the Validation Data: {mape(validation_labels, validation_predictions):.2f}%")


# -----------------------------
# RECONSTRUCT WARENGRUPPE FROM DUMMIES
# WG_2 ... WG_6 exist, WG_1 is the reference group
# -----------------------------
def reconstruct_wg(df):
    wg_cols = [c for c in df.columns if c.startswith("WG_")]
    
    # Default group = 1 (reference)
    wg = np.ones(len(df), dtype=int)
    
    for col in wg_cols:
        group_number = int(col.split("_")[1])
        wg[df[col] == 1] = group_number
    
    return wg


training_wg = reconstruct_wg(training_features)
validation_wg = reconstruct_wg(validation_features)


# -----------------------------
# BUILD DATAFRAMES FOR GROUPED EVALUATION
# -----------------------------
train_df = pd.DataFrame({
    "Warengruppe": training_wg,
    "y_true": training_labels,
    "y_pred": training_predictions
})

val_df = pd.DataFrame({
    "Warengruppe": validation_wg,
    "y_true": validation_labels,
    "y_pred": validation_predictions
})


# -----------------------------
# MAPE PER WARENGRUPPE
# -----------------------------


mape_val_wg = (
    val_df
    .groupby("Warengruppe")
    .apply(lambda x: mape(x["y_true"], x["y_pred"]))
    .reset_index(name="MAPE_Validation")
)

mape_per_wg = (
    mape_val_wg.sort_values("Warengruppe")
)

print("\nMAPE per product category (Warengruppe):")
print(mape_per_wg)


239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step
MAPE on the Training Data: 6957365970.34%
MAPE on the Validation Data: 4494038732.43%

MAPE per product category (Warengruppe):
   Warengruppe  MAPE_Validation
0            1     4.494039e+09


## Comparative Analysis

We compare the pooled linear regression in MF_20251113 with the MLP neural network from MF_neural_net_estimation. The linear model is a log1p-OLS with calendar effects (weekend, holidays, Kieler Woche), Fourier seasonality terms, lagged revenue features, and Warengruppe dummies, plus a group-wise bias correction after back-transforming predictions. This gives strong interpretability (coefficients by feature) and stable behavior, but it is limited to additive linear effects in log space and may miss higher-order interactions.

The neural network uses the same prepared feature set (stored in pickle files) and learns nonlinear interactions through hidden layers and batch normalization. It is typically more flexible for complex, interaction-heavy effects (e.g., weather x calendar x product group), but it is less interpretable and more sensitive to data scaling and training choices. We evaluate both models with MAPE on training/validation and can also compare MSE to align with the NN loss. The final choice balances interpretability and operational stability (linear model) against potential accuracy gains from nonlinear effects (neural network).

In [6]:
# Comparative analysis: baseline_model (Temperatur + KielerWoche) vs neural net
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    denom = np.maximum(np.abs(y_true), eps)
    return 100.0 * np.mean(np.abs(y_true - y_pred) / denom)

# -----------------------------
# Baseline model from baseline_model.ipynb
# Umsatz ~ Temperatur + KielerWoche
# -----------------------------
sales = pd.read_csv('/workspaces/TeamCPH/data/umsatzdaten_gekuerzt.csv', parse_dates=['Datum'])
wetter = pd.read_csv('/workspaces/TeamCPH/data/wetter1.csv', parse_dates=['Datum'])
kiwo = pd.read_csv('/workspaces/TeamCPH/data/kiwo.csv', parse_dates=['Datum'])

sales_daily = (
    sales
    .groupby(['Datum', 'Warengruppe'], as_index=False)['Umsatz']
    .sum()
)

baseline_df = sales_daily.merge(wetter, on='Datum', how='left')
baseline_df = baseline_df.merge(kiwo, on='Datum', how='left')

if 'KielerWoche' in baseline_df.columns:
    baseline_df['KielerWoche'] = baseline_df['KielerWoche'].fillna(0).astype(int)
else:
    baseline_df['KielerWoche'] = 0

baseline_features = ['Temperatur', 'KielerWoche']
baseline_df = baseline_df[['Umsatz'] + baseline_features].replace([np.inf, -np.inf], np.nan).dropna()

X_base = baseline_df[baseline_features]
y_base_log = np.log1p(baseline_df['Umsatz'])

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
    X_base, y_base_log, test_size=0.2, random_state=42
)

baseline_model = LinearRegression()
baseline_model.fit(X_train_b, y_train_b)

y_pred_base_log = baseline_model.predict(X_test_b)
y_pred_base = np.expm1(y_pred_base_log)
y_true_base = np.expm1(y_test_b)

mae_base = mean_absolute_error(y_true_base, y_pred_base)
rmse_base = np.sqrt(mean_squared_error(y_true_base, y_pred_base))
mape_base = mape(y_true_base, y_pred_base)
mse_base = mean_squared_error(y_true_base, y_pred_base)

# -----------------------------
# Neural net (saved model + validation pickle files)
# -----------------------------
subdirectory = 'pickle_data'
validation_features = pd.read_pickle(f'{subdirectory}/validation_features.pkl')
validation_labels = pd.read_pickle(f'{subdirectory}/validation_labels.pkl')

nn_model = load_model('python_model.h5', compile=False)
nn_model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
nn_pred = nn_model.predict(validation_features).reshape(-1)
nn_true = np.asarray(validation_labels).reshape(-1)

mae_nn = mean_absolute_error(nn_true, nn_pred)
rmse_nn = np.sqrt(mean_squared_error(nn_true, nn_pred))
mape_nn = mape(nn_true, nn_pred)
mse_nn = mean_squared_error(nn_true, nn_pred)

comparison = pd.DataFrame({
    'Model': ['Baseline Linear (Temp + KielerWoche)', 'Neural Net (MLP)'],
    'Dataset': ['Baseline test split (20%)', 'Pickle validation set'],
    'MAE': [mae_base, mae_nn],
    'RMSE': [rmse_base, rmse_nn],
    'MAPE_%': [mape_base, mape_nn],
    'MSE': [mse_base, mse_nn],
})

print(comparison.round(3))

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
                                  Model                    Dataset      MAE  \
0  Baseline Linear (Temp + KielerWoche)  Baseline test split (20%)  105.101   
1                      Neural Net (MLP)      Pickle validation set   36.624   

      RMSE        MAPE_%        MSE  
0  147.101  6.382900e+01  21638.589  
1   63.636  9.902875e+09   4049.522  
